# 4. Fine-Tuning de BERT avec LoRA

## 4.1. Introduction

Ce notebook passe aux modèles de Deep Learning en utilisant **BERT (Bidirectional Encoder Representations from Transformers)**. Pour rendre le fine-tuning plus efficace, nous utilisons la technique **LoRA (Low-Rank Adaptation)**, qui permet d'adapter le modèle pré-entraîné en n'entraînant qu'un très petit nombre de paramètres supplémentaires.

**Étapes Principales :**
- Installation des bibliothèques nécessaires.
- Chargement et échantillonnage des données.
- Tokenisation des textes pour BERT.
- Configuration et application de LoRA au modèle BERT.
- Entraînement avec le `Trainer` de Hugging Face.
- Évaluation et sauvegarde du modèle adapté.

## 4.2. Installation des dépendances

In [ ]:
!pip install peft transformers datasets accelerate bitsandbytes evaluate -q

## 4.3. Importation des bibliothèques

In [ ]:
import torch
import pandas as pd
import evaluate
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType

## 4.4. Chargement et Préparation des Données

In [ ]:
# Charger le jeu de données complet
data = pd.read_csv('../data/balanced_subset.csv')

# Échantillonner 30% des données pour un fine-tuning plus rapide
sampled_data = data.sample(frac=0.3, random_state=42)

# Mapper les labels textuels en entiers numériques
label_map = {"Positive": 2, "Neutral": 1, "Negative": 0}
sampled_data['rating'] = sampled_data['rating'].map(label_map)

# Supprimer les lignes où le texte est manquant (au cas où)
sampled_data.dropna(subset=['cleaned_text'], inplace=True)

# Division en ensembles d'entraînement et de validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    sampled_data['cleaned_text'].tolist(), 
    sampled_data['rating'].tolist(), 
    test_size=0.2, 
    random_state=42,
    stratify=sampled_data['rating'].tolist()
)

print(f"Taille de l'ensemble d'entraînement : {len(train_texts)}")
print(f"Taille de l'ensemble de validation : {len(val_texts)}")

## 4.5. Tokenisation

In [ ]:
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokeniser les textes
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

## 4.6. Création des Datasets Hugging Face

In [ ]:
# Créer des objets Dataset pour le Trainer
train_dataset = Dataset.from_dict({
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"],
    "labels": train_labels,
})

val_dataset = Dataset.from_dict({
    "input_ids": val_encodings["input_ids"],
    "attention_mask": val_encodings["attention_mask"],
    "labels": val_labels,
})

## 4.7. Configuration du Modèle avec LoRA

In [ ]:
# Définir le device (GPU si disponible)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Utilisation du device : {device}")

# Charger le modèle BERT pré-entraîné
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Configurer LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Tâche : Classification de Séquence
    inference_mode=False,       # Activer le mode entraînement
    r=8,                        # Rang de la matrice de décomposition (rank)
    lora_alpha=16,              # Facteur d'échelle alpha
    lora_dropout=0.1,           # Taux de dropout pour les couches LoRA
)

# Appliquer PEFT (LoRA) au modèle et le déplacer sur le bon device
model = get_peft_model(model, lora_config).to(device)

# Afficher les paramètres entraînables pour vérifier que LoRA est bien appliqué
model.print_trainable_parameters()

## 4.8. Entraînement du Modèle

In [ ]:
# Charger la métrique de précision
accuracy = evaluate.load("accuracy")

# Fonction pour calculer les métriques pendant l'évaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results_bert",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    logging_dir="./logs_bert",
    logging_steps=10,
    fp16=True,  # Activer la précision mixte pour accélérer l'entraînement sur GPU
)

# Initialiser le Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Lancer l'entraînement
trainer.train()

## 4.9. Évaluation Finale

In [ ]:
# Évaluer le modèle sur l'ensemble de validation après l'entraînement
eval_results = trainer.evaluate()

# Afficher le rapport d'évaluation
print("Résultats de l'évaluation finale :")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")

## 4.10. Sauvegarde du Modèle

Sauvegarde du modèle fine-tuné et du tokenizer pour une utilisation ultérieure.

In [ ]:
# Définir le chemin de sauvegarde
output_path = "../models/bert_lora"

# Sauvegarder le modèle et le tokenizer
model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)

print(f"Modèle sauvegardé dans {output_path}")